In [6]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

import tensorflow as tf
from gym_derk.envs import DerkEnv
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvPerso, EnvTensorOne
import tf_agents
from tf_agents.specs import tensor_spec
from tf_agents.policies import py_tf_eager_policy
from tf_agents.drivers import py_driver

from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network

In [37]:
tf.config.list_physical_devices('GPU')

[]

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
reward= {
    "damageEnemyStatue": 4,
    "damageEnemyUnit": 2,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 0,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -1,
    "healEnemy": -1,
    "fallDamageTaken": -10,
    "statueDamageTaken": 0,
    "manualBonus": 0,
    "victory": 100,
    "loss": -100,
    "tie": 0,
    "teamSpirit": 0.5,
    "timeScaling": 1,
}
args = {"home_team":[
    {
        'primaryColor': '#00ff00',
        'slots': ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
    },
    {
        'primaryColor': '#00ff00',
        'slots': ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
    },
    {
        'primaryColor': '#00ff00',
        'slots': ['Pistol', 'HealingGland', 'FrogLegs']
    },
],
"away_team":[
            {
                'primaryColor': '#00ff00',
                'slots': ['Pistol', 'HealingGland', 'FrogLegs']
            },
            {
                'primaryColor': '#00ff00',
                'slots':
                ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
            },
            {
                'primaryColor': '#00ff00',
                'slots': ['Pistol', 'HealingGland', 'FrogLegs']
            },
        ]}


try:
    env.close()
finally:
    env = EnvTensorOne(reward_function = reward, turbo_mode=True, **args)

In [9]:
# Define the network for the agent.
actor_net = actor_distribution_network.ActorDistributionNetwork(
    env.observation_tensor_spec(),
    env.action_tensor_spec())

value_net = value_network.ValueNetwork(
    env.observation_tensor_spec())

# Define the keras optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define the train step counter.
train_step_counter = tf.Variable(0)

# Create the PPO agent.
agent = ppo_agent.PPOAgent(
    env.time_step_spec(),
    env.action_tensor_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    train_step_counter=train_step_counter)

# Initialize the agent.
agent.initialize()

In [10]:
env.time_step_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(64,), dtype=tf.float32, name='observation', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'reward': BoundedTensorSpec(shape=(), dtype=tf.float32, name='reward', minimum=array(0., dtype=float32), maximum=array(inf, dtype=float32)),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [11]:
agent.collect_data_spec

_TupleWrapper(Trajectory(
{'action': BoundedTensorSpec(shape=(15,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(64,), dtype=tf.float32, name='observation', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'policy_info': DictWrapper({'dist_params': DictWrapper({'loc': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_loc'), 'scale': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_scale')})}),
 'reward': BoundedTensorSpec(shape=(), dtype=tf.float32, name='reward', minimum=array(0., dtype=float32), maximum=array(inf, dtype=float32)),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')}))

In [12]:
batch_size = 1
replay_buffer_max_length = 1000
initial_collect_steps = 100

In [13]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer_capacity = 1000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=batch_size,
    max_length=replay_buffer_capacity)

In [11]:
# class ReplayBufferDebugger(tf_uniform_replay_buffer.TFUniformReplayBuffer):
#     def add_batch(self, items):
#         print(items)
#         super().add_batch(items)

In [15]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=32,
    num_steps=10).prefetch(3)

iterator = iter(dataset)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


2023-07-08 09:45:18.698933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]


In [43]:
#import trajectory
from tf_agents.trajectories import trajectory

for _ in range(1):
    time_step = env.reset()
    while not time_step.is_last():
        action_step = agent.collect_policy.action(time_step)
        next_time_step = env.step(action_step.action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        batch = tf.nest.map_structure(lambda t: tf.expand_dims(t, 0), traj)
        replay_buffer.add_batch(batch)
        time_step = next_time_step


done


In [25]:
f"{2.1567:.2f}"

'2.16'

In [41]:
for _ in range(20):
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience)

In [31]:
train_loss

LossInfo(loss=<tf.Tensor: shape=(), dtype=float32, numpy=27.682693>, extra=PPOLossInfo(policy_gradient_loss=<tf.Tensor: shape=(), dtype=float32, numpy=-0.10904628>, value_estimation_loss=<tf.Tensor: shape=(), dtype=float32, numpy=14.743065>, l2_regularization_loss=<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, entropy_regularization_loss=<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, kl_penalty_loss=<tf.Tensor: shape=(), dtype=float32, numpy=13.048674>, clip_fraction=<tf.Tensor: shape=(), dtype=float32, numpy=0.0>))

In [40]:
def compute_avg_return(env, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    obs = env.reset()
    episode_return = 0.0
    done = False
    
    while not done:
      action_step = policy.action(obs)
      obs, reward, done, info = env.step(action_step.action)
      episode_return += reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  print(avg_return)
  return avg_return

In [17]:
iterations = 1000
batch_size = 32
learning_rate = 0.001
log_interval = 10
nb_episodes = 2

In [ ]:
# # Reset the train step.
# agent.train_step_counter.assign(0)

# # Evaluate the agent's policy once before training.
# avg_return = compute_avg_return(env, agent.policy, 2)
# returns = [avg_return]

# # Reset the environment.
# time_step = env.reset()

# # Create a driver to collect experience.
# collect_driver = dynamic_step_driver.DynamicStepDriver(
#     env,
#     agent.collect_policy,
#     observers=None,
#     num_steps=32) 

In [ ]:
# class Agent:
#     def __init__(self, shape_of_model: tuple):
#         nb_outputs = 15
#         self.model = keras.models.Sequential()
#         self.model.add(layers.Input(shape=(64,)))
#         for n in shape_of_model:
#             self.model.add(layers.Dense(units=n, activation='relu'))
#         self.model.add(layers.Dense(units=nb_outputs, activation='linear'))
    
#     def action(self, state: np.ndarray):
#         prediction = self.model.predict(state)
#         # min max scaling
        
#         return self.model.predict(state)
    


In [ ]:
# class AgentManager:
#     def __init__(self, env, number_of_agents):
#         self.env = env
#         self.agents = [Agent(shape_of_model=(15,15)) for i in range(number_of_agents)]
            
#     def take_action(self, observation):
#         return [agent.action(observation) for agent in self.agents]   
        
        